In [1]:
import os
import pandas as pd
from tqdm import tqdm 
#from sentence_transformers import SentenceTransformer
import time
from lxml import etree
import pathlib
import re
from bs4 import BeautifulSoup
import time

In [2]:
#model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [3]:
def calculate_embedding(text_list):
    return [model.encode(text) for text in text_list]

def extract_heading_text(xml_content):
    try:
        # Parsear el contenido XML para crear un árbol de elementos
        root = etree.fromstring(xml_content)
        
        # Lista para guardar textos de las etiquetas 'heading'
        heading_texts = []        
        # Recorrer todos los elementos y extraer textos de las etiquetas 'heading'
        for element in root.iter('heading'):
            # Agregar el texto de la etiqueta 'heading' a la lista
            heading_texts.append(element.text.strip() if element.text else "")
        
        return heading_texts
    except etree.XMLSyntaxError:
        print("Error al parsear xml")
        return []  
    
def extract_all_xml_tags(xml_content):
    try:
        # Parsear el contenido XML para crear un árbol de elementos
        root = etree.fromstring(xml_content)
        # Inicializar una lista para guardar solo los nombres de las etiquetas
        all_tags = []
        # Recorrer todos los elementos en el árbol XML
        for element in root.iter():
            all_tags.append(element.tag)
    
        return all_tags
    
    except etree.XMLSyntaxError:
        print("Error al procesar")
        return []

In [4]:
from lxml import etree
def parse_xml_from_bytes(data_bytes):
    try:
        # Parsear el XML directamente desde bytes
        root = etree.fromstring(data_bytes)
        # Para extraer todo el texto concatenado de los elementos XML
        return ''.join(root.xpath('//text()'))
    except Exception as e:
        print(f"Error al parsear XML: {e}")
        return None

In [5]:
import re
import xml.etree.ElementTree as ET

def extract_text(xml_string):
    # Remove the initial double quotes and escaped newlines
    xml_string = xml_string.strip('"').replace('\\n', '')
    
    # Replace double double quotes with single double quotes
    xml_string = xml_string.replace('""', '"')
    
    # Use regex to fix XML declaration if it's malformed
    xml_string = re.sub(r'(<\?xml[^>]+>)', r'\1', xml_string)
    
    # Parse XML
    root = ET.fromstring(xml_string)
    
    # Extract text from all elements
    texts = []
    for elem in root.iter():
        if elem.text:
            texts.append(elem.text.strip())
    
    # Join all extracted texts with spaces
    extracted_text = ' '.join(texts)
    return extracted_text

In [7]:
path = "/export/data_ml4ds/NextProcurement/PLACE/Septiembre2024" 
output_path = "/export/data_ml4ds/NextProcurement/PLACE/processed_all_feb_19"

all_files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.parq')]

### Filter by "tecnicas"
palabras_clave = ['tecnicas']
regex = '|'.join(palabras_clave)

df_all = []
for file in tqdm(all_files, total=len(all_files)):
    df = pd.read_parquet(file)

    start_time = time.time()
    df = df[df['doc_name'].str.contains(regex, na=False, case=False)]

    #Cuidado que los campo de "alternative_lang" y "translated_content" para los documentos no traducidos puede tener el valor None / null,
    #o puede ser un string vacío. Para evitar cualquier problema recomendamos mirar expresamente que "alternative_lang" sea "ca", "gl", o "eu" y usar "translated_content" solo en estos casos."
    
    if "translated_content" in df.columns:
        
        ddf = df.loc[df['translated_content'].notnull()]
        df_translated = ddf.loc[df['alternative_lang'].isin(['gl', 'ca', 'eu'])]
        indices_in_df_cat = df_translated.index
        df_es = ddf[~ddf.index.isin(df_translated)] # Filter df to get rows that are not in df_translated
        
        df_translated['extracted'] = df_translated['translated_content'].apply(extract_text)
        df_translated["translated"] = ["yes"] * len(df_translated)
        
        df_es['extracted'] = df_es['content'].apply(parse_xml_from_bytes)
        df_es["translated"] = ["no"] * len(df_es)
        df_es_translated = pd.concat([df_translated,df_es])
        print(f"Adding {len(df_es_translated)} new rows to final df")

    else:
        #df['extracted_tags'] = df['content'].apply(extract_all_xml_tags)
        #df['texto_heading'] = df['content'].apply(extract_heading_text)
        #df['extracted'] = df['content'].apply(parse_xml_from_bytes)
        #df["translated"] = ["no"] * len(df)
        print("No debería entrar aquí")
    
    df_es_translated = df_es_translated[['procurement_id', 'doc_name', 'extracted', 'alternative_lang', 'translated']]
  
    end_time = time.time()
    print(f"Archivo procesado: {file} en {end_time - start_time} segundos")
    df_all.append(df_es_translated)
    
    # Construir el nombre del nuevo archivo conservando el nombre original pero cambiando la ruta
    new_file_path = os.path.join(output_path, os.path.basename(file))
    # Guardar el DataFrame en la nueva ruta
    df_es_translated.to_parquet(new_file_path)
    
df_all = pd.concat(df_all)

  0%|          | 0/17 [00:00<?, ?it/s]/tmp/ipykernel_4023695/1565256794.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_translated['extracted'] = df_translated['translated_content'].apply(extract_text)
/tmp/ipykernel_4023695/1565256794.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_translated["translated"] = ["yes"] * len(df_translated)


Adding 31380 new rows to final df
Archivo procesado: /export/data_ml4ds/NextProcurement/PLACE/Septiembre2024/procurements_file_11_containing_252661_docs_mark_fixed.parq en 35.400670289993286 segundos


  6%|▌         | 1/17 [03:15<52:12, 195.77s/it]/tmp/ipykernel_4023695/1565256794.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_translated['extracted'] = df_translated['translated_content'].apply(extract_text)
/tmp/ipykernel_4023695/1565256794.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_translated["translated"] = ["yes"] * len(df_translated)


Adding 31441 new rows to final df
Archivo procesado: /export/data_ml4ds/NextProcurement/PLACE/Septiembre2024/procurements_file_13_containing_250951_docs_mark_fixed.parq en 39.07849717140198 segundos


 12%|█▏        | 2/17 [06:54<52:17, 209.17s/it]/tmp/ipykernel_4023695/1565256794.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_translated['extracted'] = df_translated['translated_content'].apply(extract_text)
/tmp/ipykernel_4023695/1565256794.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_translated["translated"] = ["yes"] * len(df_translated)


...

 88%|████████▊ | 15/17 [53:23<07:15, 217.94s/it]

Adding 4750 new rows to final df
Archivo procesado: /export/data_ml4ds/NextProcurement/PLACE/Septiembre2024/procurements_file_9_containing_250416_docs_mark_fixed.parq en 10.61481785774231 segundos


 94%|█████████▍| 16/17 [56:09<03:22, 202.37s/it]/tmp/ipykernel_4023695/1565256794.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_translated['extracted'] = df_translated['translated_content'].apply(extract_text)
/tmp/ipykernel_4023695/1565256794.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_translated["translated"] = ["yes"] * len(df_translated)


Adding 31426 new rows to final df
Archivo procesado: /export/data_ml4ds/NextProcurement/PLACE/Septiembre2024/procurements_file_6_containing_251105_docs_mark_fixed.parq en 44.56652760505676 segundos


100%|██████████| 17/17 [59:50<00:00, 211.19s/it]


In [20]:
df_all[df_all.translated == "yes"].iloc[0].extracted

'Servicios ¡ Aigues de Barberá Empresa Municipal S A C Cireumvalación. 14 Tfno. 93 72946 02 08210 - Barbera del Valles e-mail: infora sabemsa.cat web: wwwsabemsa.cat PLIEGO DE PRESCRIPCIONES TECNICAS DEL ACUERDO MARCO DE SUMINISTRO DE MATERIALES PARA OBRA CIVIL Servicios ¡ Aigues de Barberá Empresa Municipal S A C Cireumvalación. 14 Tfno. 93 72946 02 08210 - Barbera del Valles e-mail: infora sabemsa.cat web: wwwsabemsa.cat Sumario 0]] - Estimación del volumen de contratación ......................--...... ... . ._. r re eee eee eee Materiales a suministrar ........................e..ee000000reemeeo D D D IDO ÉÉÉ E Z C EÉ EG e I eee Características generales de los artículos de todos los lOts........................reeeeceeceececi e LOTE 1: características específicas de los HORMIGONES..................eeececmencccccccererer eeec D < A O A D INA- A Z ANE A m cocccecocncccecacaceecaceeaaaccanaccacacacacaaccecacccena % 6. LOTE2: características específicas de CIMENTOS, MORTEROS . MATERIAL